# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [94]:
# Details Student 1:
student_first = 'Alon Bril 316265008'

# Details Student 2:
student_second = 'Anastasia Hamandritov 321924433'

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [95]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [96]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [97]:
# word net installation:

# unmark if you want to use and need to install
!pip install wn
!python -m wn download omw-he:1.4


Cached file found: C:\Users\nasth\.wn_data\downloads\7ecf10e89326bc0ac26ad94b40fe60a7b6ac3302

Checking C:\Users\nasth\AppData\Local\Temp\tmpfkshid24\omw-he\omw-he.xml
Skipping omw-he:1.4 (Hebrew Wordnet); already added



In [98]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [99]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
!pip install hebrew_tokenizer

In [100]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [101]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [102]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [103]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

### Part 1: Split the data to X_train, X_test, y_train

In [104]:
X_train=df_train.story
X_test=df_test.story
y_train=df_train.gender

### Part 2: Test different variations
We will try different variations in order to find the best prediction

### Variation 1: K-Nearest Neighbors (KNeighborsClassifier)

#### CountVectorizer

In [105]:
# n_neighbors=13
text_clf_knn_count = Pipeline([
    ('vect', CountVectorizer()),
    ('norm', preprocessing.Normalizer()),
    ('clf_knn', KNeighborsClassifier(n_neighbors=13)),
])

# Fit and evaluate similar to before
text_clf_knn_count.fit(X_train, y_train)
knn_predictions_count = text_clf_knn_count.predict(X_train)
f1_score(y_train, knn_predictions_count,average='macro')

Pipeline(steps=[('vect', CountVectorizer()), ('norm', Normalizer()),
                ('clf_knn', KNeighborsClassifier(n_neighbors=13))])

0.5720510229856024

In [106]:
# n_neighbors=5
text_clf_knn_count = Pipeline([
    ('vect', CountVectorizer()),
    ('norm', preprocessing.Normalizer()),
    ('clf_knn', KNeighborsClassifier(n_neighbors=5)),
])

# Fit and evaluate similar to before
text_clf_knn_count.fit(X_train, y_train)
knn_predictions_count = text_clf_knn_count.predict(X_train)
f1_score(y_train, knn_predictions_count,average='macro')

Pipeline(steps=[('vect', CountVectorizer()), ('norm', Normalizer()),
                ('clf_knn', KNeighborsClassifier())])

0.6898546152430528

#### TfidfVectorizer

In [107]:
text_clf_knn_tf_idf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf_knn', KNeighborsClassifier(n_neighbors=5)),
])

# Fit and evaluate similar to before
text_clf_knn_tf_idf.fit(X_train, y_train)
knn_predictions_tf_idf = text_clf_knn_tf_idf.predict(X_train)
f1_score(y_train, knn_predictions_tf_idf,average='macro')

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf_knn', KNeighborsClassifier())])

0.7264728437083776

##### Summary of the best result:
n_neighbors = 5 | Vectorization: TF-IDF

### Variation 2: Decision Tree Classifier (DecisionTreeClassifier)

#### CountVectorizer

In [108]:
# max_features=20, max_depth=10
text_clf_dt_count = Pipeline([
    ('vect', CountVectorizer(max_features=20)),
    ('norm', preprocessing.Normalizer()),
    ('clf_dt', DecisionTreeClassifier(max_depth=10)),
])

# Fit and evaluate similar to before
text_clf_dt_count.fit(X_train, y_train)
dt_predictions_count = text_clf_dt_count.predict(X_train)
f1_score(y_train, dt_predictions_count,average='macro')

Pipeline(steps=[('vect', CountVectorizer(max_features=20)),
                ('norm', Normalizer()),
                ('clf_dt', DecisionTreeClassifier(max_depth=10))])

0.9392190495409142

In [109]:
# max_features=3, max_depth=10
text_clf_dt_count = Pipeline([
    ('vect', CountVectorizer(max_features=3)),
    ('norm', preprocessing.Normalizer()),
    ('clf_dt', DecisionTreeClassifier(max_depth=10)),
])

# Fit and evaluate similar to before
text_clf_dt_count.fit(X_train, y_train)
dt_predictions_count = text_clf_dt_count.predict(X_train)
f1_score(y_train, dt_predictions_count,average='macro')

Pipeline(steps=[('vect', CountVectorizer(max_features=3)),
                ('norm', Normalizer()),
                ('clf_dt', DecisionTreeClassifier(max_depth=10))])

0.7247213232650127

In [110]:
# max_features=3, max_depth=15
text_clf_dt_count = Pipeline([
    ('vect', CountVectorizer(max_features=3)),
    ('norm', preprocessing.Normalizer()),
    ('clf_dt', DecisionTreeClassifier(max_depth=15)),
])

# Fit and evaluate similar to before
text_clf_dt_count.fit(X_train, y_train)
dt_predictions_count = text_clf_dt_count.predict(X_train)
f1_score(y_train, dt_predictions_count,average='macro')

Pipeline(steps=[('vect', CountVectorizer(max_features=3)),
                ('norm', Normalizer()),
                ('clf_dt', DecisionTreeClassifier(max_depth=15))])

0.7978675428424172

##### TfidfVectorizer

In [111]:
# max_features=20, max_depth=10
text_clf_dt_tf_idf = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=20)),
    ('clf_dt', DecisionTreeClassifier(max_depth=10)),
])

# Fit and evaluate similar to before
text_clf_dt_tf_idf.fit(X_train, y_train)
dt_predictions_tf_idf = text_clf_dt_tf_idf.predict(X_train)
f1_score(y_train, dt_predictions_tf_idf,average='macro')

Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=20)),
                ('clf_dt', DecisionTreeClassifier(max_depth=10))])

0.9486088677571264

In [112]:
# max_features=3, max_depth=10
text_clf_dt_tf_idf = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=3)),
    ('clf_dt', DecisionTreeClassifier(max_depth=10)),
])

# Fit and evaluate similar to before
text_clf_dt_tf_idf.fit(X_train, y_train)
dt_predictions_tf_idf = text_clf_dt_tf_idf.predict(X_train)
f1_score(y_train, dt_predictions_tf_idf,average='macro')

Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=3)),
                ('clf_dt', DecisionTreeClassifier(max_depth=10))])

0.6838868996172367

In [113]:
# max_features=3, max_depth=15
text_clf_dt_tf_idf = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=3)),
    ('clf_dt', DecisionTreeClassifier(max_depth=15)),
])

# Fit and evaluate similar to before
text_clf_dt_tf_idf.fit(X_train, y_train)
dt_predictions_tf_idf = text_clf_dt_tf_idf.predict(X_train)
f1_score(y_train, dt_predictions_tf_idf,average='macro')

Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=3)),
                ('clf_dt', DecisionTreeClassifier(max_depth=15))])

0.808847737164003

##### Summary of the best result:
max_features=3 | max_depth=15 | Vectorization: TF-IDF

### Variation 3: Multinomial Naive Bayes Classifier (MultinomialNB)

#### CountVectorizer

In [114]:
# max_features=100, norm l1
text_clf_nb_count = Pipeline([
    ('vect', CountVectorizer(max_features=100)),
    ('norm', preprocessing.Normalizer()),
    ('clf_dt', MultinomialNB()),
])

# Fit and evaluate similar to before
text_clf_nb_count.fit(X_train, y_train)
nb_predictions_count = text_clf_nb_count.predict(X_train)
f1_score(y_train, nb_predictions_count,average='macro')

Pipeline(steps=[('vect', CountVectorizer(max_features=100)),
                ('norm', Normalizer()), ('clf_dt', MultinomialNB())])

0.4329819277108434

In [115]:
# max_features=20, norm = l2
text_clf_nb_count = Pipeline([
    ('vect', CountVectorizer(max_features=20)),
    ('clf_dt', MultinomialNB()),
])

# Fit and evaluate similar to before
text_clf_nb_count.fit(X_train, y_train)
nb_predictions_count = text_clf_nb_count.predict(X_train)
f1_score(y_train, nb_predictions_count,average='macro')

Pipeline(steps=[('vect', CountVectorizer(max_features=20)),
                ('clf_dt', MultinomialNB())])

0.5983821503177918

##### TfidfVectorizer

In [116]:
# max_features=20
text_clf_nb_tf_idf = Pipeline([
    ('vect', TfidfVectorizer(max_features=20)),
    ('clf_dt', MultinomialNB()),
])

# Fit and evaluate similar to before
text_clf_nb_tf_idf.fit(X_train, y_train)
nb_predictions_tf_idf = text_clf_nb_tf_idf.predict(X_train)
f1_score(y_train, nb_predictions_tf_idf,average='macro')

Pipeline(steps=[('vect', TfidfVectorizer(max_features=20)),
                ('clf_dt', MultinomialNB())])

0.4329819277108434

##### Summary of the best result:
max_features=20 | norm = l2 | Vectorization: Count

### Variation 4: Support Vector Machine (SVM) Classifier (Linear SVC)

#### CountVectorizer

In [117]:
# max_features=100
text_clf_svm_count = Pipeline([
    ('vect', CountVectorizer(max_features=100)),
    ('norm', preprocessing.Normalizer()),
    ('clf_svm', LinearSVC()),
])

# Fit and evaluate similar to before
text_clf_svm_count.fit(X_train, y_train)
svm_predictions_count = text_clf_svm_count.predict(X_train)
f1_score(y_train, svm_predictions_count,average='macro')

Pipeline(steps=[('vect', CountVectorizer(max_features=100)),
                ('norm', Normalizer()), ('clf_svm', LinearSVC())])

0.6257256845956047

In [118]:
# max_features=500
text_clf_svm_count = Pipeline([
    ('vect', CountVectorizer(max_features=500)),
    ('norm', preprocessing.Normalizer()),
    ('clf_svm', LinearSVC()),
])

# Fit and evaluate similar to before
text_clf_svm_count.fit(X_train, y_train)
svm_predictions_count = text_clf_svm_count.predict(X_train)
f1_score(y_train, svm_predictions_count,average='macro')

Pipeline(steps=[('vect', CountVectorizer(max_features=500)),
                ('norm', Normalizer()), ('clf_svm', LinearSVC())])

0.8368568691844553

##### TfidfVectorizer

In [119]:
# max_features=100
text_clf_svm_tf_idf = Pipeline([
    ('vect', TfidfVectorizer(max_features=100)),
    ('norm', preprocessing.Normalizer()),
    ('clf_svm', LinearSVC()),
])

# Fit and evaluate similar to before
text_clf_svm_tf_idf.fit(X_train, y_train)
svm_predictions_tf_idf = text_clf_svm_tf_idf.predict(X_train)
f1_score(y_train, svm_predictions_tf_idf,average='macro')

Pipeline(steps=[('vect', TfidfVectorizer(max_features=100)),
                ('norm', Normalizer()), ('clf_svm', LinearSVC())])

0.6629482071713146

In [120]:
# max_features=500
text_clf_svm_tf_idf = Pipeline([
    ('vect', TfidfVectorizer(max_features=500)),
    ('norm', preprocessing.Normalizer()),
    ('clf_svm', LinearSVC()),
])

# Fit and evaluate similar to before
text_clf_svm_tf_idf.fit(X_train, y_train)
svm_predictions_tf_idf = text_clf_svm_tf_idf.predict(X_train)
f1_score(y_train, svm_predictions_tf_idf,average='macro')

Pipeline(steps=[('vect', TfidfVectorizer(max_features=500)),
                ('norm', Normalizer()), ('clf_svm', LinearSVC())])

0.9047639123102866

##### Summary of the best result:
max_features=500 | Vectorization: TF-IDF

### Variation 5: Multi-Layer Perceptron Classifier (MLPClassifier)

#### CountVectorizer

In [121]:
# max_features=20, norm l1
text_clf_mlp_count = Pipeline([
    ('vect', CountVectorizer(max_features=20)),
    ('norm', preprocessing.Normalizer()),
    ('clf_mlp', MLPClassifier()),
])

# Fit and evaluate similar to before
text_clf_mlp_count.fit(X_train, y_train)
mlp_predictions_count = text_clf_mlp_count.predict(X_train)
f1_score(y_train, mlp_predictions_count,average='macro')

Pipeline(steps=[('vect', CountVectorizer(max_features=20)),
                ('norm', Normalizer()), ('clf_mlp', MLPClassifier())])

0.6107311995836586

In [122]:
# max_features=20, norm l2
text_clf_mlp_count = Pipeline([
    ('vect', CountVectorizer(max_features=20)),
    ('clf_mlp', MLPClassifier()),
])

# Fit and evaluate similar to before
text_clf_mlp_count.fit(X_train, y_train)
mlp_predictions_count = text_clf_mlp_count.predict(X_train)
f1_score(y_train, mlp_predictions_count,average='macro')

Pipeline(steps=[('vect', CountVectorizer(max_features=20)),
                ('clf_mlp', MLPClassifier())])

0.8622705122780198

##### TfidfVectorizer

In [123]:
text_clf_mlp_tf_idf = Pipeline([
    ('vect', TfidfVectorizer(max_features=20)),
    ('clf_mlp', MLPClassifier()),
])

# Fit and evaluate similar to before
text_clf_mlp_tf_idf.fit(X_train, y_train)
mlp_predictions_tf_idf = text_clf_mlp_tf_idf.predict(X_train)
f1_score(y_train, mlp_predictions_tf_idf,average='macro')

Pipeline(steps=[('vect', TfidfVectorizer(max_features=20)),
                ('clf_mlp', MLPClassifier())])

0.6163326300383336

##### Summary of the best result:
max_features=20 | norm = L2 | Vectorization: Count

### Part 3: Predict
From our previous testing, we see that out best option is Multi-Layer Perceptron Classifier with Counter. Let's predict!

#### FIt with Hyper parameters using GridSearchCV

In [124]:
# max_features=20, norm l2
text_clf_mlp_count = Pipeline([
    ('vect', CountVectorizer(max_features=20)),
    ('clf_mlp', MLPClassifier()),
])

param_grid = {
    'clf_mlp__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'clf_mlp__activation': ['logistic', 'relu'],
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(text_clf_mlp_count, param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(max_features=20)),
                                       ('clf_mlp', MLPClassifier())]),
             param_grid={'clf_mlp__activation': ['logistic', 'relu'],
                         'clf_mlp__hidden_layer_sizes': [(50,), (100,),
                                                         (50, 50), (100, 50)]})

Best parameters: {'clf_mlp__activation': 'logistic', 'clf_mlp__hidden_layer_sizes': (100,)}


#### Cross validation

In [125]:
# Use cross_val_score to evaluate the model with best parameters
best_mlp = grid_search.best_estimator_
cv_scores = cross_val_score(best_mlp, X_train, y_train, cv=5)  # 5-fold cross-validation
print("Cross-validation scores:", cv_scores)
print("Mean accuracy:", cv_scores.mean())

Cross-validation scores: [0.7615894  0.75496689 0.76821192 0.75333333 0.77333333]
Mean accuracy: 0.7622869757174393


#### Predict

In [126]:
predicted = best_mlp.predict(X_test)

#### Adding the predicted gender to our test data

In [133]:
test_predicted = df_test.copy()
test_predicted['gender'] = 0
for i in range(0,len(test_predicted['test_example_id'])):
    test_predicted['gender'][i] = predicted[i]

df_test_predicted = pd.DataFrame(columns = ['test_example_id','predicted_category'])

df_test_predicted['test_example_id'] = df_test['test_example_id']
df_test_predicted['predicted_category'] = test_predicted['gender']

In [140]:
df_test_predicted

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m
...,...,...
318,318,m
319,319,m
320,320,m
321,321,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [141]:
df_test_predicted.to_csv('classification_results.csv',index=False)